# An end-to-end workflow to efficiently compress and deploy DNN classifiers on SoC/FPGA.

## DNN training and compression

In [1]:
# Import libraries

import os
import csv
import numpy as np
from numpy import array
import time
import glob

import tensorflow as tf 

from tensorflow.keras.models import *
from tensorflow.keras.layers import *

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import categorical_crossentropy as logloss
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Lambda
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input

import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn import decomposition
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE, MDS
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, auc, roc_auc_score, r2_score, mean_squared_error, mean_absolute_error

import seaborn as sn
import pandas as pd

import keras_tuner as kt
from qkeras import *

from qkeras import QActivation
from qkeras import QDense, QConv2DBatchnorm


#pip install scikit-image
import skimage.data
import skimage.transform
from skimage import io

import shutil, sys

from itertools import cycle

from keras.applications.vgg16 import preprocess_input

# Custom functions
from src.distillationClassKeras import *

# Plot confusion matrix
from src.confMatrix import *

from src.studentCompression import *
from src.studentOptimization import *
from src.teacherOptimization import *
from src.loadDataset import *


2023-06-16 14:25:15.119556: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# Path Vivado HLS 
os.environ['PATH'] = '/tools/Xilinx2019/Vivado/2019.2/bin:' + os.environ['PATH']
os.environ['PATH']

seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

import tensorflow as tf
print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    except RuntimeError as e:
        print(e)


GPUs:  1


2023-06-16 14:25:19.462873: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-06-16 14:25:19.886726: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-16 14:25:19.886864: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1650 with Max-Q Design computeCapability: 7.5
coreClock: 1.155GHz coreCount: 14 deviceMemorySize: 3.82GiB deviceMemoryBandwidth: 149.04GiB/s
2023-06-16 14:25:19.886879: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-06-16 14:25:19.930310: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-06-16 14:25:19.930417: I tensorflow/s

In [3]:
# Definitions

# Dataset path
ROOT_PATH = "/home/ro/kaleido/research/repo/pestControl/dataset"
# Class labels 
classLabels=['0', '1', '2']
# Input shape
ROWS, COLS =  80, 80 

# Teacher model
# 0: train teacher from scratch, 1: pre-trained model
TEACHER_OP = 0

# Number of iterations for BO
N_ITERATIONS = 2

# Type of input - 1: 1D signal or 2: 2D signal
D_SIGNAL = 1


### Load dataset 

In [ ]:
if D_SIGNAL == '1':
    # Load 1D signal dataset
    images_train, images_validation, images_test, y_train, y_test = loadDataset_2D(ROOT_PATH, classLabels, ROWS, COLS)

else:
    # Load 2D signal dataset
    signal_train, signal_validation, signal_test, y_train, y_val, y_test = loadDataset_1D(ROOT_PATH, classLabels)


### Teacher model optimization

In [ ]:
# Grab the best hyperparameters

def bestHPBO_computation(bestHP_BO, CONV_VAR, FC_VAR, UPPER_CONV, UPPER_FC):

    bestHP = []
    # Grab hyper-params
    for i in range (1,UPPER_CONV):
        bestHP.append(bestHP_BO.get(CONV_VAR + str(i)))
    for j in range (1, UPPER_FC):
        bestHP.append(bestHP_BO.get(FC_VAR + str(j)))
    
    return bestHP

In [ ]:
# Decide if optimize a teacher architecture or load a pre-trained network as teacher

if TEACHER_OP == 0:
    # optimize teacher architecture
    print("Teacher optimization")
    bestHP_BO_teacher = teacherBO(images_train, y_train, images_test, y_test, N_ITERATIONS)
    # Grab the best hyperparameters
        
else: 

    # Load pre-trained model
   # teacher_baseline = load_model('/home/ro/kaleido/research/repo/Methodology_MLclassificationSoC/ML_src/models/CNN/teacher_NEW_v2_ok.h5')  #RPIC modified
    teacher_baseline = load_model('models/preTrained/teacher_NEW_v2_ok.h5')  #RPIC modified
    teacher_baseline.summary()

In [ ]:
# Grab the best hyperparameters for student training
lr = bestHP_BO_teacher.get('learning_rate')
CONV_VAR = 'conv_'
FC_VAR = 'fc'
UPPER_CONV = 4
UPPER_FC = 3

bestHP_BO_teacher = bestHPBO_computation(bestHP_BO_teacher, CONV_VAR, FC_VAR, UPPER_CONV, UPPER_FC)

teacherModel = studentCompression(bestHP_BO_teacher, images_train, y_train, teacher_baseline, lr)
teacherModel.summary()

# Save model 
teacherModel.save("models/teacherFP.h5")

### . Student model optimization

In [ ]:
# Bayesian optimization for student architecture
bestHP_BO = studentBO(images_train, y_train, images_test, y_test, teacher_baseline, N_ITERATIONS)

In [ ]:
# Grab the best hyperparameters for student training
CONV_VAR = 'conv_'
FC_VAR = 'fc'
UPPER_CONV = 9
UPPER_FC = 4
lr = bestHP_BO.get('learning_rate')

bestHP = bestHPBO_computation(bestHP_BO, CONV_VAR, FC_VAR, UPPER_CONV, UPPER_FC)



### . Student training

In [ ]:
# Training to obtain compressed model
model = studentCompression(bestHP, images_train, y_train, teacher_baseline, lr)

# Model summary
model.summary()

In [ ]:
# Save model 
model.save("models/distilled_student.h5")

In [ ]:
# Plot confusion matrix for accuracy evaluation
confusionMatrixPlot(model, images_train, y_train)


# Integration with a hardware synthesis tool for ML

In [ ]:
import hls4ml

# Based on the tutorials provided by hls4ml

hls4ml.model.optimizer.OutputRoundingSaturationMode.layers = ['Activation']
hls4ml.model.optimizer.OutputRoundingSaturationMode.rounding_mode = 'AP_RND'
hls4ml.model.optimizer.OutputRoundingSaturationMode.saturation_mode = 'AP_SAT'

hls_config = hls4ml.utils.config_from_keras_model(model, granularity='name')

for Layer in hls_config['LayerName'].keys():
    hls_config['LayerName'][Layer]['Strategy'] = 'Latency'
    hls_config['LayerName'][Layer]['ReuseFactor'] = 1

hls_config['LayerName']['output_softmax']['Strategy'] = 'Stable'

cfg = hls4ml.converters.create_config(backend='Vivado')
cfg['IOType']     = 'io_stream' # Must set this if using CNNs!
cfg['HLSConfig']  = hls_config
cfg['KerasModel'] = model
cfg['OutputDir']  = 'model/'
cfg['XilinxPart'] = 'xc7z020-clg484-1'  # PYNQ-Z1 or Zedboard: xc7z020-clg484-1
  
hls_model = hls4ml.converters.keras_to_hls(cfg)
hls_model.compile()

hls_model.build(csim=False, export=True)


# Hardware assessment framework

After generating IP core though High-level Synthesis, the developer should integrate it with the corresponding design. Source files are in the folder: assessmentFramework